In [1]:
import dask.bag as db
import dask.array as da
import dask.dataframe as df
import math
import pandas as pd
import numpy as np
import requests

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


Client Scheduler: tcp://127.0.0.1:42821 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 33.68 GB


Helper function that displays all the functions that can be invoked on an object.

In [3]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [4]:
def get_split_words(word_bag):    
    split_words = word_bag.str.split()
    stripped_words = split_words.flatten().map(lambda x: x.strip())
    word_array = stripped_words.filter(lambda x: x!= "")
    return word_array

In [5]:
def get_filtered_words(words):
    stopwords = open("/home/dsp_team_gregory/gregory-p1/data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [6]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    split_words = get_split_words(word_bag)
    long_words = split_words.filter(lambda x: len(x) < 3)
#     words_without_punctuations = long_words.map(puntuation_stripper)
    return get_filtered_words(long_words)    
#     return long_words

In [7]:
def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    large_bag = db.concat(unique_words)
    frequencies = large_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], round(math.log((len(words_by_document) + 1)/x[1]), 10)))
    return idf

In [8]:
def get_TF_IDF(tf, idf):
    tf = np.asarray(tf.frequencies().compute())
    tf_idf = {}
    for idf_item in idf: 
        col_name = str(int(idf_item[0], 16))
        tf_idf[col_name] = idf_item[1]
        for i in range(len(tf)):
            tf_item = tf[i]
            if idf_item[0] == tf_item[0]:
                tf_idf[col_name] = float(idf_item[1]) * float(tf_item[1])
                np.delete(tf, i)
                break
    return tf_idf

In [9]:
set_name = "X_test.txt"
# set_name = "X_train.txt"
# url = "/home/dsp_team_gregory/gregory-p1/data/train/" + set_name
url = 'https://storage.googleapis.com/uga-dsp/project1/files/' + set_name
file_names = pd.read_csv(url, header=None)[0].to_numpy()

In [10]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

21:40:00


In [11]:
all_file_words = []
for file_name in file_names:
#     file_name = "/home/dsp_team_gregory/gregory-p1/data/train/" + file_name + ".bytes"
    file_name = "https://storage.googleapis.com/uga-dsp/project1/data/bytes/" + file_name + ".bytes"
    all_file_words.append(get_words_for_document(file_name))
idf = get_IDF(all_file_words)

In [12]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

21:40:08


In [ ]:
idf = np.asarray(idf.topk(256, key=0).compute())

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

21:50:02


In [ ]:
cols = np.arange(255, -1, -1).astype(str)
pd_df = pd.DataFrame(columns=cols)
i = 0
for file_words in all_file_words:
    print("working on file number %d" % i)
    i += 1
    tf_idf_bag_array = get_TF_IDF(file_words, idf)
    pd_df = pd_df.append(tf_idf_bag_array, ignore_index=True)

working on file number 0
working on file number 1
working on file number 2
working on file number 3
working on file number 4
working on file number 5
working on file number 6
working on file number 7
working on file number 8
working on file number 9
working on file number 10
working on file number 11
working on file number 12
working on file number 13
working on file number 14
working on file number 15
working on file number 16
working on file number 17
working on file number 18
working on file number 19
working on file number 20
working on file number 21
working on file number 22
working on file number 23
working on file number 24
working on file number 25
working on file number 26
working on file number 27
working on file number 28
working on file number 29
working on file number 30
working on file number 31
working on file number 32
working on file number 33
working on file number 34
working on file number 35
working on file number 36
working on file number 37
working on file number

working on file number 308
working on file number 309
working on file number 310
working on file number 311
working on file number 312
working on file number 313
working on file number 314
working on file number 315
working on file number 316
working on file number 317
working on file number 318
working on file number 319
working on file number 320
working on file number 321
working on file number 322
working on file number 323
working on file number 324
working on file number 325
working on file number 326
working on file number 327
working on file number 328
working on file number 329
working on file number 330
working on file number 331
working on file number 332
working on file number 333
working on file number 334
working on file number 335
working on file number 336
working on file number 337
working on file number 338
working on file number 339
working on file number 340
working on file number 341
working on file number 342
working on file number 343
working on file number 344
w

working on file number 612
working on file number 613
working on file number 614
working on file number 615
working on file number 616
working on file number 617
working on file number 618
working on file number 619
working on file number 620
working on file number 621
working on file number 622
working on file number 623
working on file number 624
working on file number 625
working on file number 626
working on file number 627
working on file number 628
working on file number 629
working on file number 630
working on file number 631
working on file number 632
working on file number 633
working on file number 634
working on file number 635
working on file number 636
working on file number 637
working on file number 638
working on file number 639
working on file number 640
working on file number 641
working on file number 642
working on file number 643
working on file number 644
working on file number 645
working on file number 646
working on file number 647
working on file number 648
w

working on file number 916
working on file number 917
working on file number 918
working on file number 919
working on file number 920
working on file number 921
working on file number 922
working on file number 923
working on file number 924
working on file number 925
working on file number 926
working on file number 927
working on file number 928
working on file number 929
working on file number 930
working on file number 931
working on file number 932
working on file number 933
working on file number 934
working on file number 935
working on file number 936
working on file number 937
working on file number 938
working on file number 939
working on file number 940
working on file number 941
working on file number 942
working on file number 943
working on file number 944
working on file number 945
working on file number 946
working on file number 947
working on file number 948
working on file number 949
working on file number 950
working on file number 951
working on file number 952
w

working on file number 1212
working on file number 1213
working on file number 1214
working on file number 1215
working on file number 1216
working on file number 1217
working on file number 1218
working on file number 1219
working on file number 1220
working on file number 1221
working on file number 1222
working on file number 1223
working on file number 1224
working on file number 1225
working on file number 1226
working on file number 1227
working on file number 1228
working on file number 1229
working on file number 1230
working on file number 1231
working on file number 1232
working on file number 1233
working on file number 1234
working on file number 1235
working on file number 1236
working on file number 1237
working on file number 1238
working on file number 1239
working on file number 1240
working on file number 1241
working on file number 1242
working on file number 1243
working on file number 1244
working on file number 1245
working on file number 1246
working on file numb

working on file number 1507
working on file number 1508
working on file number 1509
working on file number 1510
working on file number 1511
working on file number 1512
working on file number 1513
working on file number 1514
working on file number 1515
working on file number 1516
working on file number 1517
working on file number 1518
working on file number 1519
working on file number 1520
working on file number 1521
working on file number 1522
working on file number 1523
working on file number 1524
working on file number 1525
working on file number 1526
working on file number 1527
working on file number 1528
working on file number 1529
working on file number 1530
working on file number 1531
working on file number 1532
working on file number 1533
working on file number 1534
working on file number 1535
working on file number 1536
working on file number 1537
working on file number 1538
working on file number 1539
working on file number 1540
working on file number 1541
working on file numb

working on file number 1800
working on file number 1801
working on file number 1802
working on file number 1803
working on file number 1804
working on file number 1805
working on file number 1806
working on file number 1807
working on file number 1808
working on file number 1809
working on file number 1810
working on file number 1811
working on file number 1812
working on file number 1813
working on file number 1814
working on file number 1815
working on file number 1816
working on file number 1817
working on file number 1818
working on file number 1819
working on file number 1820
working on file number 1821
working on file number 1822
working on file number 1823
working on file number 1824
working on file number 1825
working on file number 1826
working on file number 1827
working on file number 1828
working on file number 1829
working on file number 1830
working on file number 1831
working on file number 1832
working on file number 1833
working on file number 1834
working on file numb

working on file number 2093
working on file number 2094
working on file number 2095
working on file number 2096
working on file number 2097
working on file number 2098
working on file number 2099
working on file number 2100
working on file number 2101
working on file number 2102
working on file number 2103
working on file number 2104
working on file number 2105
working on file number 2106
working on file number 2107
working on file number 2108
working on file number 2109
working on file number 2110
working on file number 2111
working on file number 2112
working on file number 2113
working on file number 2114
working on file number 2115
working on file number 2116
working on file number 2117
working on file number 2118
working on file number 2119
working on file number 2120
working on file number 2121
working on file number 2122
working on file number 2123
working on file number 2124
working on file number 2125
working on file number 2126
working on file number 2127
working on file numb

working on file number 2386
working on file number 2387
working on file number 2388
working on file number 2389
working on file number 2390
working on file number 2391
working on file number 2392
working on file number 2393
working on file number 2394
working on file number 2395
working on file number 2396
working on file number 2397
working on file number 2398
working on file number 2399
working on file number 2400
working on file number 2401
working on file number 2402
working on file number 2403
working on file number 2404
working on file number 2405
working on file number 2406
working on file number 2407
working on file number 2408
working on file number 2409
working on file number 2410
working on file number 2411
working on file number 2412
working on file number 2413
working on file number 2414
working on file number 2415
working on file number 2416
working on file number 2417
working on file number 2418
working on file number 2419
working on file number 2420
working on file numb

working on file number 2679
working on file number 2680
working on file number 2681
working on file number 2682
working on file number 2683
working on file number 2684
working on file number 2685
working on file number 2686
working on file number 2687
working on file number 2688
working on file number 2689
working on file number 2690
working on file number 2691
working on file number 2692
working on file number 2693
working on file number 2694
working on file number 2695
working on file number 2696
working on file number 2697
working on file number 2698
working on file number 2699
working on file number 2700
working on file number 2701
working on file number 2702
working on file number 2703
working on file number 2704
working on file number 2705
working on file number 2706
working on file number 2707
working on file number 2708
working on file number 2709
working on file number 2710
working on file number 2711
working on file number 2712
working on file number 2713
working on file numb

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

22:38:56


In [ ]:
csv_name = "/home/dsp_team_gregory/gregory-p1/data/" + set_name + ".csv"
pd_df.to_csv(csv_name)
csv_name

'/home/dsp_team_gregory/gregory-p1/data/X_test.txt.csv'

In [ ]:
tf_idf_df = df.from_pandas(pd_df, npartitions=1)
final_dataframe = tf_idf_df.compute()

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

22:38:57


In [ ]:
final_dataframe

,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,1.49871,1.94561,1.87907,1.23264,1.78203,1.39067,5.05372,1.21132,1.22675,1.26204,...,1.25542,1.26865,1.23117,1.22602,1.26204,1.20838,1.28923,1.23117,1.55678,3.31349
1,2.45351,0.29548,0.524904,0.76957,0.820438,0.501286,1.95144,0.643146,0.374127,0.332231,...,0.132304,0.245498,0.122014,0.123484,0.229327,0.406468,0.272694,0.15362,0.515252,18.5329
2,45.6619,8.77692,0.352876,9.52885,0.301048,0.182286,0.830023,8.87027,0.391033,0.261669,...,0.244763,9.29291,7.29511,432.474,17.8802,11.8184,8.39397,12.4829,7.67144,34.8975
3,29.8736,1.76994,2.91233,2.95259,2.88476,0.499081,3.66446,1.79787,0.905549,1.08122,...,2.40426,5.22528,1.69203,1.64645,1.95884,3.54355,2.11687,2.17273,3.97207,87.6148
4,2.94598,0.251378,0.299945,0.321205,0.416835,0.237413,0.794703,0.371187,0.265344,0.249173,...,0.281514,0.649761,0.343991,0.342521,0.331496,0.967291,0.736494,0.560823,1.51047,25.523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716,2.62845,2.23888,3.30711,2.32929,3.40636,2.24697,9.0655,2.23594,2.28886,2.28592,...,2.29107,2.35869,2.17714,2.29915,2.25799,2.29327,2.34987,2.33444,2.25946,4.75634
2717,72.0779,0.31165,0.420143,0.794561,0.362801,0.260198,1.08315,0.746784,0.271224,0.288129,...,2.65932,1.78464,2.45204,3.67144,3.90151,7.67732,3.58986,2.55494,2.55715,145.463
2718,3.83756,0.424844,0.561294,0.739434,0.565705,0.357957,1.57175,0.660786,0.410143,0.46674,...,0.441014,0.984932,0.538037,0.558618,0.804851,1.04226,0.61448,0.785741,1.4186,20.319
2719,41.7104,5.20838,0.320897,6.23447,0.223856,0.127159,0.688742,6.19552,0.249908,0.191841,...,0.229327,12.8975,6.77178,5.90886,7.99633,729.93,5.73319,6.20434,6.02132,31.3965


In [ ]:
%store final_dataframe

Stored 'final_dataframe' (DataFrame)
